### Бэггинг и случайный лес

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from sklearn import ensemble, model_selection, datasets, tree 
from sklearn.model_selection import cross_val_score

import numpy as np
import pandas as pd

In [3]:
# Загружаем датасет digits с помощью функции load_digits из sklearn.datasets 

digits = datasets.load_digits()

In [4]:
# Подготавливаем матрицу признаков X и ответов на обучающей выборке y

X = digits.data
y = digits.target
X.shape, y.shape

((1797, 64), (1797,))

In [5]:
# Пишем функцию записи ответов в текстовый файл

def write_answer(answer, n):
  with open('C2W4_answer_{}.txt'.format(n), 'w') as f:
    f.write(str(answer))

### Задание 1. 
Создайте DecisionTreeClassifier с настройками по умолчанию и измерьте качество его работы с помощью cross_val_score. 
Эта величина и будет ответом в пункте 1.

In [6]:
dtc = tree.DecisionTreeClassifier()
dtc.fit(X, y)

DecisionTreeClassifier()

In [8]:
res1 = cross_val_score(dtc, X, y, cv=10).mean()
print(res1)
write_answer(res1, 1)

0.8235940409683427


### Задание 2. 
Воспользуйтесь BaggingClassifier из sklearn.ensemble, чтобы обучить бэггинг над DecisionTreeClassifier. Используйте в BaggingClassifier параметры по умолчанию, задав только количество деревьев равным 100. 
Качество классификации новой модели - ответ в пункте 2.

In [9]:
bc1 = ensemble.BaggingClassifier(n_estimators=100)
bc1.fit(X, y)

BaggingClassifier(n_estimators=100)

In [10]:
res2 = cross_val_score(bc1, X, y, cv=10).mean()
print(res2)
write_answer(res2, 2)

0.9248261949099937


### Задание 3. 
Теперь изучите параметры BaggingClassifier и выберите их такими, чтобы каждый базовый алгоритм обучался не на всех d признаках, а на sqrt{d} случайных признаков. 
Качество работы получившегося классификатора - ответ в пункте 3.

In [15]:
sqrt_d = int(np.sqrt(X.shape[1]))
bc2 = ensemble.BaggingClassifier(n_estimators=100, max_features=sqrt_d)
bc2.fit(X, y)

BaggingClassifier(max_features=8, n_estimators=100)

In [16]:
res3 = cross_val_score(bc2, X, y, cv=10).mean()
print(res3)
write_answer(res3, 3)

0.9337895716945995


### Задание 4.
Наконец, давайте попробуем выбирать случайные признаки не один раз на все дерево, а при построении каждой вершины дерева. Сделать это несложно: нужно убрать выбор случайного подмножества признаков в BaggingClassifier и добавить его в DecisionTreeClassifier. Попробуйте выбирать опять же sqrt{d} признаков. 
Качество полученного классификатора на контрольной выборке и будет ответом в пункте 4.

In [17]:
tree = tree.DecisionTreeClassifier(max_features = sqrt_d)
bc3 = ensemble.BaggingClassifier(base_estimator=tree, n_estimators=100)
bc3.fit(X, y)

BaggingClassifier(base_estimator=DecisionTreeClassifier(max_features=8),
                  n_estimators=100)

In [18]:
res4 = cross_val_score(bc3, X, y, cv=10).mean()
print(res4)
write_answer(res4, 4)

0.9432309124767224


### Задание 5. 
Полученный в пункте 4 классификатор - бэггинг на рандомизированных деревьях (в которых при построении каждой вершины выбирается случайное подмножество признаков и разбиение ищется только по ним). Это в точности соответствует алгоритму Random Forest, поэтому почему бы не сравнить качество работы классификатора с RandomForestClassifier из sklearn.ensemble. Сделайте это, а затем изучите, как качество классификации на данном датасете зависит от количества деревьев, количества признаков, выбираемых при построении каждой вершины дерева, а также ограничений на глубину дерева. Для наглядности лучше построить графики зависимости качества от значений параметров, но для сдачи задания это делать не обязательно. 

In [94]:
rfc = ensemble.RandomForestClassifier()
rfc.fit(X, y)

RandomForestClassifier()

In [95]:
res = cross_val_score(rfc, X, y, cv=10).mean()
print(res)

0.9454562383612662


### Выводы:
- 2) При очень маленьком числе деревьев (5, 10, 15), случайный лес работает хуже, чем при большем числе деревьев.
- 3) С ростом количества деревьев в случайном лесе, в какой-то момент деревьев становится достаточно для высокого качества классификации, а затем качество существенно не меняется.
- 4) При большом количестве признаков (для данного датасета - 40, 50) качество классификации становится хуже, чем при малом количестве признаков (5, 10). Это связано с тем, что чем меньше признаков выбирается в каждом узле, тем более различными получаются деревья (ведь деревья сильно неустойчивы к изменениям в обучающей выборке), и тем лучше работает их композиция.
- 7) При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса заметно хуже, чем без ограничений, т.к. деревья получаются недообученными. С ростом глубины качество сначала улучшается, а затем не меняется существенно, т.к. из-за усреднения прогнозов и различий деревьев их переобученность в бэггинге не сказывается на итоговом качестве (все деревья преобучены по-разному, и при усреднении они компенсируют переобученность друг-друга).

In [96]:
res5 = '2 3 4 7'
write_answer(res5, 5)